<a href="https://colab.research.google.com/github/b08202011/Mlproject/blob/main/%E5%85%88normalize%2Bsoftmax%2B%E7%AE%97%E6%9C%9F%E6%9C%9B%E5%80%BC%2BsoftL1loss%2Bconstraint%2Blr0_001%2Bmaxpoollayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
!gdown 15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
!gdown 187gijxd4T5yuZe_g2K9UNepx7MT1CnAu

Downloading...
From: https://drive.google.com/uc?id=1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
To: /content/train.csv
100% 23.3M/23.3M [00:00<00:00, 56.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
To: /content/test.csv
100% 8.88M/8.88M [00:00<00:00, 49.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=187gijxd4T5yuZe_g2K9UNepx7MT1CnAu
To: /content/sample_submission.csv
100% 50.5k/50.5k [00:00<00:00, 107MB/s]


In [2]:
import csv
import numpy as np
from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
with open('train.csv',newline='') as csvfile:
  train = csv.reader(csvfile)
  train = list(train)

In [3]:
pick = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,23]
#可以選要用那些特徵，0是我們要predict的
#只選數值或true fale 但且沒有選id
feature = []
for i in range(1,len(train)):
  feature.append([])
  for j in pick:
    feature[i-1].append(train[i][j])

for i in range(len(feature)):
  for j in range(len(feature[i])):
    if(feature[i][j]!='' and feature[i][j]!='False' and feature[i][j]!= 'True'):
      feature[i][j] = float(feature[i][j])
    elif(feature[i][j] == 'False'):
      feature[i][j] = 0
    elif(feature[i][j] == 'True'):
      feature[i][j] = 1
    else:
      feature[i][j] = np.nan
print(len(feature))


with open('test.csv',newline='') as csvfile:         ###load testdata
  testcsv = csv.reader(csvfile)
  testcsv = list(testcsv)
  
test = []
picktest = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,22]

for i in range(1,len(testcsv)):
  test.append([])
  for j in picktest:
    test[i-1].append(testcsv[i][j])
for i in range(len(test)):
  for j in range(len(test[i])):
    if(test[i][j]!='' and test[i][j]!='False' and test[i][j]!= 'True'):
      test[i][j] = float(test[i][j])
    elif(test[i][j] == 'False'):
      test[i][j] = 0
    elif(test[i][j] == 'True'):
      test[i][j] = 1
    else:
      test[i][j] = np.nan


17170


In [4]:
!pip install ycimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv
from sklearn.feature_selection import SelectKBest
# For Progress Bar
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_regression
# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import normalize

In [6]:
feature = np.array(feature)
test = np.array(test)
from ycimpute.imputer import knnimput
xtrain, ytrain = feature[:,1:], feature[:,0]
print(xtrain.shape[0])
print(test.shape[0])
data = np.concatenate((xtrain,test))
print(data.shape)

data_np = torch.from_numpy(data)
mean, std, var = torch.mean(data_np), torch.std(data_np), torch.var(data_np)
data_np = (data_np-mean)/std
data_np = np.array(data.tolist())
print(data_np)



#imp_median.fit(feature)
#feature = imp_median.transform(feature)
##可以修改這裡 使用不同的data填補方式

17170
6315
(23485, 16)
[[ 2.73359449e-04  3.00000000e+00 -2.51760000e+01 ...  0.00000000e+00
   0.00000000e+00  3.07000000e+03]
 [ 1.84220009e-01  7.00000000e+00             nan ...  0.00000000e+00
              nan  1.22000000e+02]
 [            nan  6.00000000e+00 -1.55960000e+01 ...  0.00000000e+00
   0.00000000e+00  1.22000000e+02]
 ...
 [ 4.26222894e-01  5.00000000e+00 -5.80754484e+00 ...  1.00000000e+00
   0.00000000e+00             nan]
 [ 2.04440254e-01             nan -1.15639636e+01 ...  1.00000000e+00
   1.00000000e+00  1.51390000e+04]
 [ 1.46792398e-02             nan             nan ...  1.00000000e+00
   1.00000000e+00  1.10000000e+01]]


In [7]:
data_np = knnimput.KNN(k=4).complete(data_np)

Imputing row 1/23485 with 1 missing, elapsed time: 155.771
Imputing row 101/23485 with 1 missing, elapsed time: 155.801
Imputing row 201/23485 with 0 missing, elapsed time: 155.833
Imputing row 301/23485 with 2 missing, elapsed time: 155.859
Imputing row 401/23485 with 1 missing, elapsed time: 155.886
Imputing row 501/23485 with 1 missing, elapsed time: 155.914
Imputing row 601/23485 with 4 missing, elapsed time: 155.944
Imputing row 701/23485 with 2 missing, elapsed time: 155.976
Imputing row 801/23485 with 2 missing, elapsed time: 156.013
Imputing row 901/23485 with 3 missing, elapsed time: 156.043
Imputing row 1001/23485 with 0 missing, elapsed time: 156.074
Imputing row 1101/23485 with 0 missing, elapsed time: 156.101
Imputing row 1201/23485 with 2 missing, elapsed time: 156.131
Imputing row 1301/23485 with 3 missing, elapsed time: 156.163
Imputing row 1401/23485 with 3 missing, elapsed time: 156.194
Imputing row 1501/23485 with 3 missing, elapsed time: 156.224
Imputing row 1601/23

In [8]:
print(data_np[3])
feature, test_data = data_np[:xtrain.shape[0],:], data_np[xtrain.shape[0]:,:]
print(feature.shape)
ytrain = np.reshape(ytrain,(ytrain.shape[0],1))
feature = np.concatenate((ytrain,feature),axis=1)
print(feature.shape[0])

[ 2.09584584e-01  7.76528498e+00 -6.25100000e+00  2.77000000e-02
  3.79641600e-03  2.11999054e-02  1.00000000e-03  3.71806581e-01
  1.50220000e+02  2.65000000e+05  2.02457327e+08  9.97035000e+05
  3.99661898e+08  1.00000000e+00  9.99999948e-01  2.43500000e+04]
(17170, 16)
17170


In [9]:
print(feature)

[[0.00000000e+00 2.73359449e-04 3.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 3.07000000e+03]
 [0.00000000e+00 1.84220009e-01 7.00000000e+00 ... 0.00000000e+00
  1.39303658e-01 1.22000000e+02]
 [0.00000000e+00 2.42020446e-02 6.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.22000000e+02]
 ...
 [1.00000000e+00 5.71787000e-01 4.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 4.51217663e-01 9.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.00000000e+00 8.25293672e-01 6.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]]


In [10]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = torch.clamp(torch.round(model(x)),0,9)
                          
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

##Dataloader

In [11]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Model

In [18]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.MaxPool1d(kernel_size=4),
            nn.BatchNorm1d(16),
                                     
        )
        self.maxpoollayer = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Dropout(0.1),
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.Sigmoid(),
            nn.MaxPool1d(kernel_size=4),
            nn.BatchNorm1d(16),
        )
        self.finallayer = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.Linear(16, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 10),
        )
        self.softmax = nn.Softmax(dim=1)
        self.fc = nn.Linear(10, 1)
    def forward(self, x):
        #x = x.view(-1, 16)
        x = self.layers(x)
        x = self.maxpoollayer(x)
        x = self.maxpoollayer(x)
        x = self.maxpoollayer(x)
        x = self.finallayer(x)
        x = self.softmax(x)
        #x = self.fc(x)
        #print(x)
        out = torch.linspace(0,9,10).to(device)
        
        #x = torch.sigmoid(x)
        #x = torch.mul(x,9)
        x = torch.mul(x,out)
        x = torch.sum(x, dim=1)
        #print(x)
        x = x.squeeze(-1) # (B, 1) -> (B)
        return x
class wideL1loss(nn.Module):
  def __init__(self):
    super(wideL1loss, self).__init__()
  def forward(self, x,y):
    loss = torch.mean(torch.sigmoid(torch.subtract(torch.clamp(torch.abs(x-y),2.5,10),2.5)))
    return loss


## Training

In [13]:
def trainer(train_data, model, config, device):

    criterion = nn.SmoothL1Loss(beta=0.5) # Define your loss function, do not modify this.
    loss2 = wideL1loss()
    criterion2 = nn.L1Loss()

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), lr= config['learning_rate'] , weight_decay=0.01, amsgrad=False) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0


    np.random.seed(config['seed'])
    splits = np.array_split(train_data, 5)
    train =  np.concatenate(splits[:0]+splits[0+1:])
    valid= splits[0]
    x_train = train[...,1:]
    x_valid = valid[...,1:]
    y_train = train[...,0]
    y_valid = valid[...,0]
    train_dataset  = COVID19Dataset(x_train, y_train)
    valid_dataset = COVID19Dataset(x_valid, y_valid)
    

    for epoch in range(n_epochs):
      lossvalid_record = []
      loss_record = []
      realloss =[]
      for i in range(1):
        model.train() # Set your model to train mode.
         
        
        
        # Pytorch data loader loads pytorch dataset into batches.
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)


        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            
            #loss = criterion(pred, y)
            l1_norm = sum(p.abs().sum()for p in model.parameters())
                
            loss = criterion(pred, y)+config['loss2']*loss2(pred, y)+config['regulizer']*l1_norm
            #print(loss)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            
            loss_record.append(loss.detach().item())
            
         
        model.eval() # Set your model to evaluation mode.
        
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = torch.clamp(torch.round(model(x)),0,9)
                #loss = criterion(pred, y)
                l1_norm = sum(p.abs().sum()for p in model.parameters()) 
                loss = criterion(pred, y)+config['regulizer']*l1_norm+config['loss2']*loss2(pred, y)
                acloss = criterion2(pred, y)
            lossvalid_record.append(loss.item())
            realloss.append(acloss.item())


        
      mean_train_loss = sum(loss_record)/len(loss_record)         ###compute CV loss
      writer.add_scalar('Loss/train', mean_train_loss, step)
      reloss = sum(realloss)/len(realloss)
      mean_valid_loss = sum(lossvalid_record)/len(lossvalid_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, L1 loss: {reloss:.4f}')
      # writer.add_scalar('Loss/valid', mean_valid_loss, step)

      if mean_valid_loss < best_loss:
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
      else: 
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return

## config

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'n_fold': 5,   # validation_size = train_size * valid_ratio
    'n_epochs': 30000,     # Number of epochs.            
    'batch_size': 128, 
    'learning_rate': 1e-3,
    'regulizer': 0,  
    'loss2'  :1.5,          
    'early_stop': 1500,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}


In [15]:
same_seed(config['seed'])

#imp_median.fit(test)
#test_data = imp_median.transform(test)
train_data= feature
#train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

x_test =  test_data
test_dataset = COVID19Dataset(x_test)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

## Training and testing

In [16]:



model = My_Model(input_dim=train_data.shape[1]-1).to(device) # put your model and data on the same computation device.
trainer(train_data, model, config, device)





Epoch [1/30000]: Train loss: 2.4884, Valid loss: 2.4103, L1 loss: 2.3634
Saving model with loss 2.410...
Epoch [2/30000]: Train loss: 2.2891, Valid loss: 2.1452, L1 loss: 2.1413
Saving model with loss 2.145...
Epoch [3/30000]: Train loss: 2.1168, Valid loss: 1.9939, L1 loss: 2.0106
Saving model with loss 1.994...
Epoch [4/30000]: Train loss: 2.0096, Valid loss: 1.9509, L1 loss: 1.9644
Saving model with loss 1.951...
Epoch [5/30000]: Train loss: 1.9490, Valid loss: 1.8935, L1 loss: 1.9182
Saving model with loss 1.894...
Epoch [6/30000]: Train loss: 1.9163, Valid loss: 1.8044, L1 loss: 1.8384
Saving model with loss 1.804...
Epoch [7/30000]: Train loss: 1.8763, Valid loss: 1.8060, L1 loss: 1.8344
Epoch [8/30000]: Train loss: 1.8414, Valid loss: 1.7909, L1 loss: 1.8214
Saving model with loss 1.791...
Epoch [9/30000]: Train loss: 1.8312, Valid loss: 1.7667, L1 loss: 1.8016
Saving model with loss 1.767...
Epoch [10/30000]: Train loss: 1.8084, Valid loss: 1.7394, L1 loss: 1.7748
Saving model 

In [17]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'Danceability'])
        for i, p in enumerate(preds):
            writer.writerow([i+17170, p])
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
print(preds)
save_pred(preds, 'sample_submission.csv')
from google.colab import files
files.download('sample_submission.csv')

100%|██████████| 50/50 [00:00<00:00, 360.25it/s]

[7. 7. 5. ... 1. 3. 8.]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>